# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Необходимо построить модель с максимально большим значением *accuracy*. Доля правильных ответов не менее 0.75.

# Постановка задачи

**Нужно разработать алгоритм, который проанализирует поведение клиентов и предложит пользователям новый тариф: «`Смарт`» или «`Ультра`».**

## Откроем и изучим файл

Произведем **импорт библиотек.**

In [1]:
import pandas as pd
import numpy as np 
import random

In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [3]:
from sklearn.metrics import accuracy_score

Запишим в переменную **csv-файл.**

In [4]:
try:
    data = pd.read_csv('C:/Users/User/Desktop/Проекты/Введение в машинное обучение/users_behavior.csv')
except:
    try:
        data = pd.read_csv('C:/Users/JERR/Desktop/Проекты/Введение в машинное обучение/users_behavior.csv')
    except:
        data = pd.read_csv('/datasets/users_behavior.csv')

Напишем функцию для того чтобы отобразить **общую информацию** по датафрейму. `Входные данные` -  датасет, `выходные данные` - таблица с количеством строк, типом данных и первые пять строк датафрейма.

In [5]:
def info(data):
    data.info()
    display(data.head(5))

info(data)    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных:** 

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Следующим шагом необходимо разбить данные на выборки.**

**Соотношение двух классов: `Смарт` и `Ультра`.**

In [6]:
data.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [7]:
def observation(label, integer):
    print(f'{label} к общему числу наблюдений равно \033[1m{round(data.is_ultra.value_counts()[integer] / data.is_ultra.count() * 100, 2)}\033[0m%.')

In [8]:
observation('Смарт', 0)
observation('Смарт', 1)

Смарт к общему числу наблюдений равно 69.35%.
Смарт к общему числу наблюдений равно 30.65%.


## Разбейте данные на выборки

Данные нужно разбить на **три части: обучающую, валидационную и тестовую.** Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении **3:1:1.**

### Разбивка вариант первый

In [9]:
data_train_valid, data_test = train_test_split(
    data, 
    test_size=0.2, 
    random_state=12345)

In [10]:
data_train, data_valid = train_test_split(
    data_train_valid, 
    test_size = 0.25,
    random_state=12345)

In [29]:
print('Размер обучающей выборки:',data_train.shape)
print('Размер валидационной выборки:',data_valid.shape)
print('Размер тестовой выборки:', data_test.shape)

Размер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выборки: (643, 5)


### Разбивка вариант второй

Добавим метод `stratify` в `train_test_split`.

Отделяем целевую переменную.

In [12]:
X = data.drop('is_ultra', axis=1)
y = data['is_ultra']

Делим данные на **тренировочную** и **валидационную** выборки.

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=12345, stratify=y)

Делим выборку на **валидационную** и **тестовую.**

In [14]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=12345, stratify=y_valid)

**P.S. Указываем stratify=y, чтобы сохранить соотношение классов при разбиении.**

## Исследуйте модели

Так как **целевой признак категориальный**, то решается задача `классификации`, в частности речь идёт о бинарной (двоичной) классификации (либо `Smart`, либо `Ultra`).

**Признаки** - `calls`, `minutes`, `messages` и `mb_used`

**Цель** - `is_ultra`

In [15]:
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']

### Модель решаюшего дерева

In [16]:
best_model_tree = None
best_result_tree = 0

In [17]:
for depth in range(2, 22, 2):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_tree.fit(features_train, target_train) 
    predictions_tree = model_tree.predict(features_valid) 
    result_tree = accuracy_score(target_valid, predictions_tree) 
    print("max_depth =", depth, "| ", end='')
    print(accuracy_score(target_valid, predictions_tree).round(4))
    if result_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result_tree

max_depth = 2 | 0.7574
max_depth = 4 | 0.7636
max_depth = 6 | 0.7574
max_depth = 8 | 0.7667
max_depth = 10 | 0.7714
max_depth = 12 | 0.7558
max_depth = 14 | 0.7574
max_depth = 16 | 0.7496
max_depth = 18 | 0.7418
max_depth = 20 | 0.7294


In [18]:
print("Лучшая модель:", best_model_tree)
print("Accuracy лучшей модели:", best_result_tree.round(2))

Лучшая модель: DecisionTreeClassifier(max_depth=10, random_state=12345)
Accuracy лучшей модели: 0.77


### Модель случайного леса

In [19]:
best_model_forest = None
best_result_forest = 0

In [20]:
for depth in range(14, 18, 2):
    for est in range(100, 500, 50):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est) 
        model_forest.fit(features_train, target_train) 
        predictions_forest = model_forest.predict(features_valid) 
        result_forest = accuracy_score(target_valid, predictions_forest) 
        print("n_estimators =", est, "|", "depth =", depth, "| ",  end='')
        print(result_forest.round(4))
        if result_forest > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result_forest

n_estimators = 100 | depth = 14 | 0.7963
n_estimators = 150 | depth = 14 | 0.7916
n_estimators = 200 | depth = 14 | 0.79
n_estimators = 250 | depth = 14 | 0.7916
n_estimators = 300 | depth = 14 | 0.79
n_estimators = 350 | depth = 14 | 0.7932
n_estimators = 400 | depth = 14 | 0.7916
n_estimators = 450 | depth = 14 | 0.7932
n_estimators = 100 | depth = 16 | 0.8009
n_estimators = 150 | depth = 16 | 0.7994
n_estimators = 200 | depth = 16 | 0.8009
n_estimators = 250 | depth = 16 | 0.7978
n_estimators = 300 | depth = 16 | 0.7978
n_estimators = 350 | depth = 16 | 0.8025
n_estimators = 400 | depth = 16 | 0.8009
n_estimators = 450 | depth = 16 | 0.7994


In [21]:
print("Лучшая модель:", best_model_forest)
print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest.round(2))

Лучшая модель: RandomForestClassifier(max_depth=16, n_estimators=350, random_state=12345)
Accuracy наилучшей модели на валидационной выборке: 0.8


### Логистическая регрессия

In [22]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
model_regression.fit(features_train, target_train) 
result_regression = model_regression.score(features_valid, target_valid).round(4)

print("Accuracy модели (логистическая регрессия) на валидационной выборке:", result_regression.round(2))

Accuracy модели (логистическая регрессия) на валидационной выборке: 0.73


### Вывод

In [23]:
print('Модель решаюшего дерева: ', best_result_tree.round(2))
print('Модель случайного леса: ', best_result_forest.round(2))
print('Логистическая регрессия: ', result_regression.round(2))

Модель решаюшего дерева:  0.77
Модель случайного леса:  0.8
Логистическая регрессия:  0.73


Заметим, что по валидационной выборке уже двух из трех моделей удалось достичь **точности равной 0.75.** Увеличим точность, обучив модели на тестовой выборке.

## Проверьте модель на тестовой выборке

### Модель решаюшего дерева

In [24]:
best_model_tree.fit(features_train, target_train) 
predictions_tree = best_model_tree.predict(features_test) 
result_tree = accuracy_score(target_test, predictions_tree).round(2) 
print('Модель решаюшего дерева: ', result_tree)

Модель решаюшего дерева:  0.77


### Модель случайного леса

In [25]:
best_model_forest.fit(features_train, target_train) 
predictions_forest = best_model_forest.predict(features_test) 
result_forest = accuracy_score(target_test, predictions_forest).round(2) 
print('Модель случайного леса: ', result_forest)

Модель случайного леса:  0.79


### Логистическая регрессия

In [26]:
model_regression.fit(features_train, target_train) 
predictions_regression = model_regression.predict(features_test)
result_regression = model_regression.score(features_test, target_test).round(2)
print('Логистическая регрессия: ', result_regression)

Логистическая регрессия:  0.76


### Вывод

In [27]:
print('Модель решаюшего дерева:' , result_tree)
print('Модель случайного леса:' , result_forest)
print('Логистическая регрессия:' , result_regression)

Модель решаюшего дерева: 0.77
Модель случайного леса: 0.79
Логистическая регрессия: 0.76


**Все модели достигли точности не меньше 0.75.**

## (бонус) Проверьте модели на адекватность

In [28]:
model_dummy = DummyClassifier(random_state=12345) 
model_dummy.fit(features_train, target_train) 
predictions_model_dummy = model_dummy.predict(features_test) 
result_dummy = accuracy_score(target_test, predictions_model_dummy).round(2) 
print('Проверка модели на адекватность:', result_dummy)

Проверка модели на адекватность: 0.58


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


## Вывод

По достигнутым моделям рекомендуется тариф "Ультра" для получения максимальной прибыли. В ходе исследования использовались три модели: решаюшего дерева, случайного леса и логистическая регрессия; с точностью не менее 0.75.